In [1]:
# Imports and page set up
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import time

playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()
await page.goto("https://cas.columbia.edu/cas/login?service=https%3a%2f%2fwww1.columbia.edu%2fsec-cgi-bin%2fcul%2fprox%2fezproxy-auth.cgi%3furl%3dezp.2aHR0cDovL3d3dy5uZXhpc3VuaS5jb20v")

<Response url='https://cas.columbia.edu/cas/login?service=https%3a%2f%2fwww1.columbia.edu%2fsec-cgi-bin%2fcul%2fprox%2fezproxy-auth.cgi%3furl%3dezp.2aHR0cDovL3d3dy5uZXhpc3VuaS5jb20v' request=<Request url='https://cas.columbia.edu/cas/login?service=https%3a%2f%2fwww1.columbia.edu%2fsec-cgi-bin%2fcul%2fprox%2fezproxy-auth.cgi%3furl%3dezp.2aHR0cDovL3d3dy5uZXhpc3VuaS5jb20v' method='GET'>>

In [2]:
await page.locator(".advanced-search").click()
await page.get_by_text("Legal").click()
await page.get_by_text("Cases").click()
await page.get_by_text("United States Cases").click()

In [3]:
await page.locator('xpath=//*[@id="3xLg9kk"]/div/div[3]/div[1]/div[1]/div[1]/div[1]/input').type("asylum")
await page.locator('xpath=//*[@id="3xLg9kk"]/div/div[3]/div[1]/div[2]/div[1]/div/div/button').click()
await page.get_by_text("Date is after").click()

In [ ]:
# Selected date to be after Jan. 1 2020. And clicked search. 

In [4]:
# To further narrow down the search results: click on "Court" on the left side
await page.locator('xpath=//*[@id="podfiltersbuttoncourt-rollup"]').click()

# To select 2nd Circuit:
await page.locator('xpath=//*[@id="refine"]/ul[2]/li[2]/ul/li[2]/label/span[1]').click()

# To narrow it down further, I select the dropdown menu for practice areas
await page.locator('[id="podfiltersbuttonssa"]').click()

# Here I click "Immigration Law"
await page.locator("span").get_by_text("Immigration Law").click()

# This further gives me a dropdown menu, where I select Asylum, Refugee, and related relief
await page.locator("span").get_by_text("Asylum, Refugees & Related Relief").click()

## In search results page now

In each individual case (after clicking in), I'm extracting the case name, a list of core terms, and opinion. And putting them into a dictionary.
<br><br>
*time for some functions??*


In [6]:
# This function gets all the core terms on each individual case page, 
# and makes the terms into a list of terms
async def get_core_terms(page):
    # Get all the content from the individual case
    all_text = await page.content()
    # Get the beginning index of the core terms, find the first <br> after the title "Core Terms"
    core_terms_tag = all_text.find('id="JUMPTO_CoreTerms"')
    core_text_start = all_text[core_terms_tag:].find("<br>") + len("<br>") + core_terms_tag
    # Get the ending index of the core terms
    core_text_end = all_text[core_text_start:].find("<br>") + core_text_start
    core_text = all_text[core_text_start:core_text_end]
    core_terms = core_text.split(",")
    return core_terms

In [8]:
# This goes into the ith result on a search result page 
# and gets the name, opinion, and terms for each case
async def case_dict(i):
    await page.locator(f".row_sr{i} >> h2 >> a").click()
    print("Scraping case:", i)
    time.sleep(2)
    html = await page.content()
    soup_doc = BeautifulSoup(html, "html.parser")
    case = {}
    case["name"] = soup_doc.find(id="SS_DocumentTitle").text
    opinion_ps = soup_doc.find(id="JUMPTO_Opinion").find_next_siblings("p")
    case["opinion"] = "".join([p.text for p in opinion_ps])
    case["terms"] = await get_core_terms(page)
    return case


In [9]:
async def next_page():
    time.sleep(1)
    await page.locator('[data-action="nextpage"]').click()

In [10]:
all_cases = []

In [11]:
case_i = 0
async def whole_page(case_i):
    print("in whole page")
    for _ in range(10):
        print("case num", case_i)
        case = await case_dict(case_i)
        print("after case")
        all_cases.append(case)
        case_i += 1
        time.sleep(2)
        await page.get_by_text("Results list").first.click()
    return case_i

        

In [12]:
async def scrape(case_i):
    print("Starting scraping...")
    for _ in range(45): #NexisUni kicks me out around page 46/47. So I will stop myself at range 45 scrape from 46 - 67 after
        print("in while")
        case_i = await whole_page(case_i)
        await next_page()

In [13]:
await scrape(case_i)

Starting scraping...
in while
in whole page
case num 0
Scraping case: 0
after case
case num 1
Scraping case: 1
after case
case num 2
Scraping case: 2
after case
case num 3
Scraping case: 3
after case
case num 4
Scraping case: 4
after case
case num 5
Scraping case: 5
after case
case num 6
Scraping case: 6
after case
case num 7
Scraping case: 7
after case
case num 8
Scraping case: 8
after case
case num 9
Scraping case: 9
after case
in while
in whole page
case num 10
Scraping case: 10
after case
case num 11
Scraping case: 11
after case
case num 12
Scraping case: 12
after case
case num 13
Scraping case: 13
after case
case num 14
Scraping case: 14
after case
case num 15
Scraping case: 15
after case
case num 16
Scraping case: 16
after case
case num 17
Scraping case: 17
after case
case num 18
Scraping case: 18
after case
case num 19
Scraping case: 19
after case
in while
in whole page
case num 20
Scraping case: 20
after case
case num 21
Scraping case: 21
after case
case num 22
Scraping case: 2

case num 185
Scraping case: 185
after case
case num 186
Scraping case: 186
after case
case num 187
Scraping case: 187
after case
case num 188
Scraping case: 188
after case
case num 189
Scraping case: 189
after case
in while
in whole page
case num 190
Scraping case: 190
after case
case num 191
Scraping case: 191
after case
case num 192
Scraping case: 192
after case
case num 193
Scraping case: 193
after case
case num 194
Scraping case: 194
after case
case num 195
Scraping case: 195
after case
case num 196
Scraping case: 196
after case
case num 197
Scraping case: 197
after case
case num 198
Scraping case: 198
after case
case num 199
Scraping case: 199
after case
in while
in whole page
case num 200
Scraping case: 200
after case
case num 201
Scraping case: 201
after case
case num 202
Scraping case: 202
after case
case num 203
Scraping case: 203
after case
case num 204
Scraping case: 204
after case
case num 205
Scraping case: 205
after case
case num 206
Scraping case: 206
after case
case num

case num 366
Scraping case: 366
after case
case num 367
Scraping case: 367
after case
case num 368
Scraping case: 368
after case
case num 369
Scraping case: 369
after case
in while
in whole page
case num 370
Scraping case: 370
after case
case num 371
Scraping case: 371
after case
case num 372
Scraping case: 372
after case
case num 373
Scraping case: 373
after case
case num 374
Scraping case: 374
after case
case num 375
Scraping case: 375
after case
case num 376
Scraping case: 376
after case
case num 377
Scraping case: 377
after case
case num 378
Scraping case: 378
after case
case num 379
Scraping case: 379
after case
in while
in whole page
case num 380
Scraping case: 380
after case
case num 381
Scraping case: 381
after case
case num 382
Scraping case: 382
after case
case num 383
Scraping case: 383
after case
case num 384
Scraping case: 384
after case
case num 385
Scraping case: 385
after case
case num 386
Scraping case: 386
after case
case num 387
Scraping case: 387
after case
case num

In [14]:
len(all_cases)

450

In [16]:
import json

In [17]:
with open('asylum_data_3.json', 'w+', encoding='utf-8') as f:
    json.dump(all_cases, f, ensure_ascii=False, indent=4)

Stop here and restart kernal. So NexisUni doesn't think I'm scraping too much at once. Why do they have a search limit?

I have tried this same code twice and NexisUni kicked me out at around the 460th case both times. So I changed the code a little bit down below to scrape the rest, in this case from the 451th case with the index 450.

#### From case 451 to the end of time

In [ ]:
# Imports and page set up
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup
import time

playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()
await page.goto("https://cas.columbia.edu/cas/login?service=https%3a%2f%2fwww1.columbia.edu%2fsec-cgi-bin%2fcul%2fprox%2fezproxy-auth.cgi%3furl%3dezp.2aHR0cDovL3d3dy5uZXhpc3VuaS5jb20v")

In [2]:
await page.locator(".advanced-search").click()
await page.get_by_text("Legal").click()
await page.get_by_text("Cases").click()
await page.get_by_text("United States Cases").click()

In [3]:
await page.locator('xpath=//*[@id="3xLg9kk"]/div/div[3]/div[1]/div[1]/div[1]/div[1]/input').type("asylum")
await page.locator('xpath=//*[@id="3xLg9kk"]/div/div[3]/div[1]/div[2]/div[1]/div/div/button').click()
await page.get_by_text("Date is after").click()

In [ ]:
# Selected date to be after Jan. 1 2020. And clicked search. 

In [4]:
# To further narrow down the search results: click on "Court" on the left side
await page.locator('xpath=//*[@id="podfiltersbuttoncourt-rollup"]').click()

# To select 2nd Circuit:
await page.locator('xpath=//*[@id="refine"]/ul[2]/li[2]/ul/li[2]/label/span[1]').click()

# To narrow it down further, I select the dropdown menu for practice areas
await page.locator('[id="podfiltersbuttonssa"]').click()

# Here I click "Immigration Law"
await page.locator("span").get_by_text("Immigration Law").click()

# This further gives me a dropdown menu, where I select Asylum, Refugee, and related relief
await page.locator("span").get_by_text("Asylum, Refugees & Related Relief").click()

In [6]:
# This function gets all the core terms on each individual case page, 
# and makes the terms into a list of terms
async def get_core_terms(page):
    # Get all the content from the individual case
    all_text = await page.content()
    # Get the beginning index of the core terms, find the first <br> after the title "Core Terms"
    core_terms_tag = all_text.find('id="JUMPTO_CoreTerms"')
    core_text_start = all_text[core_terms_tag:].find("<br>") + len("<br>") + core_terms_tag
    # Get the ending index of the core terms
    core_text_end = all_text[core_text_start:].find("<br>") + core_text_start
    core_text = all_text[core_text_start:core_text_end]
    core_terms = core_text.split(",")
    return core_terms

In [8]:
# This goes into the ith result on a search result page 
# and gets the name, opinion, and terms for each case
async def case_dict(i):
    await page.locator(f".row_sr{i} >> h2 >> a").click()
    print("Scraping case:", i)
    time.sleep(2)
    html = await page.content()
    soup_doc = BeautifulSoup(html, "html.parser")
    case = {}
    case["name"] = soup_doc.find(id="SS_DocumentTitle").text
    opinion_ps = soup_doc.find(id="JUMPTO_Opinion").find_next_siblings("p")
    case["opinion"] = "".join([p.text for p in opinion_ps])
    case["terms"] = await get_core_terms(page)
    return case


In [9]:
async def next_page():
    time.sleep(1)
    await page.locator('[data-action="nextpage"]').click()

In [10]:
# This is a list of all cases unscraped when NexisUni became a hater
all_cases_from_451 = []
case_i = 450

In [11]:
async def whole_page(case_i):
    print("in whole page")
    for _ in range(10):
        try:
            print("case num", case_i)
            case = await case_dict(case_i)
            print("after case")
            all_cases_from_451.append(case)
            case_i += 1
            time.sleep(1)
            await page.get_by_text("Results list").first.click()
        except:
            break
    return case_i

In [12]:
async def scrape(case_i):
    print("Starting scraping...")
    for _ in range(22): # 22 is the number of pages left
        print("in while")
        case_i = await whole_page(case_i)
        await next_page()


In [13]:
# starting this from page 46:
await scrape(case_i)


Starting scraping...
in while
in whole page
case num 450
Scraping case: 450
after case
case num 451
Scraping case: 451
after case
case num 452
Scraping case: 452
after case
case num 453
Scraping case: 453
after case
case num 454
Scraping case: 454
after case
case num 455
Scraping case: 455
after case
case num 456
Scraping case: 456
after case
case num 457
Scraping case: 457
after case
case num 458
Scraping case: 458
after case
case num 459
Scraping case: 459
after case
in while
in whole page
case num 460
Scraping case: 460
after case
case num 461
Scraping case: 461
after case
case num 462
Scraping case: 462
after case
case num 463
Scraping case: 463
after case
case num 464
Scraping case: 464
after case
case num 465
Scraping case: 465
after case
case num 466
Scraping case: 466
after case
case num 467
Scraping case: 467
after case
case num 468
Scraping case: 468
after case
case num 469
Scraping case: 469
after case
in while
in whole page
case num 470
Scraping case: 470
after case
case nu

Scraping case: 630
after case
case num 631
Scraping case: 631
after case
case num 632
Scraping case: 632
after case
case num 633
Scraping case: 633
after case
case num 634
Scraping case: 634
after case
case num 635
Scraping case: 635
after case
case num 636
Scraping case: 636
after case
case num 637
Scraping case: 637
after case
case num 638
Scraping case: 638
after case
case num 639
Scraping case: 639
after case
in while
in whole page
case num 640
Scraping case: 640
after case
case num 641
Scraping case: 641
after case
case num 642
Scraping case: 642
after case
case num 643
Scraping case: 643
after case
case num 644
Scraping case: 644
after case
case num 645
Scraping case: 645
after case
case num 646
Scraping case: 646
after case
case num 647
Scraping case: 647
after case
case num 648
Scraping case: 648
after case
case num 649
Scraping case: 649
after case
in while
in whole page
case num 650
Scraping case: 650
after case
case num 651
Scraping case: 651
after case
case num 652
Scraping

TimeoutError: Timeout 30000ms exceeded.
=========================== logs ===========================
waiting for locator("[data-action=\"nextpage\"]")
============================================================

In [15]:
len(all_cases_from_451)

217

In [16]:
import json

In [17]:
with open('asylum_data_3.json',"r") as f:
    all_cases_before = json.load(f)

In [18]:
# combine all the scraped cases into one big json file
all_cases = all_cases_before + all_cases_from_451

In [19]:
len(all_cases)

667

In [20]:
with open('asylum_data_complete.json', 'w+', encoding='utf-8') as f:
    json.dump(all_cases, f, ensure_ascii=False, indent=4)